# Face Area Calculations

Computing and working with face areas is important for the analysis of unstructured grids, with many algorithms and workflows requiring them. This section will showcase the different face area calculation options provided with UXarray:

1. Calculate Total Face Area
2. Options for `Grid.calculate_total_face_area` Function
3. Getting Area of Individual Faces
4. Calculate Area of a Single Triangle in Cartesian Coordinates
5. Calculate Area from Multiple Faces in Spherical Coordinates
6. Demonstrate Area Correction at Line of Constant Lattitude

In [1]:
import uxarray as ux
import numpy as np

We will be using the `outCSne30.ug` grid file, which is encoded in the UGRID convention.

In [2]:
base_path = "../../test/meshfiles/"
grid_path = base_path + "/ugrid/outCSne30/outCSne30.ug"

ugrid = ux.open_grid(grid_path)
ugrid

<uxarray.Grid>
Original Grid Type: UGRID
Grid Dimensions:
  * n_node: 5402
  * n_face: 5400
  * n_max_face_nodes: 4
  * n_nodes_per_face: (5400,)
Grid Coordinates (Spherical):
  * node_lon: (5402,)
  * node_lat: (5402,)
Grid Coordinates (Cartesian):
Grid Connectivity Variables:
  * face_node_connectivity: (5400, 4)
Grid Descriptor Variables:
  * n_nodes_per_face: (5400,)

## 1. Calculate Total Face Area
We can calculate the total face area by calling the function `Grid.calculate_total_face_area()`. Since our dataset lies on the unit sphere, our expected area is 4π, which is approximately 12.56

In [3]:
t4_area = ugrid.calculate_total_face_area()
t4_area

np.float64(12.566370614678554)

## 2. Options for `Grid.calculate_total_face_area` Function

By default, `Grid.calculate_total_face_area` uses a Triangular Quadrature Rule and an Order of 4. However, you can specify the Quadrature Rule and Order as follows:


**Order:**

       1 to 10              for gaussian

       1, 4, 8, 10 and 12   for triangular


In [4]:
t1_area = ugrid.calculate_total_face_area(quadrature_rule="triangular", order=1)
t1_area

np.float64(12.571403993719983)

For the result above, notice that the area is slightly different than the first calculation we made.

Now we use Triangular Quadrature Rule and Order 1

Using a lower order is faster, but at the sacrifice of accuracy.

Generally, gaussian quadrature rule is more accurate than the triangular quadrature rule. Additionally, a higher order comes at the cost of computation time, but produces a more accurate result. See `uxarray/grid/area.py` file and function `get_gauss_quadratureDG` for details on quadrature points and weights.

## 3. Getting Area of Individual Faces

We can access the Grid attribute `Grid.face_area` to access the individual face areas. If you have not run a face area calculation yet, it will run the `Grid.compute_face_areas` and cache the value.



In [5]:
ugrid.face_areas

<xarray.DataArray 'face_areas' (n_face: 5400)> Size: 43kB
array([0.00211174, 0.00211221, 0.00210723, ..., 0.00210723, 0.00211221,
       0.00211174])
Dimensions without coordinates: n_face
Attributes:
    cf_role:    face_areas
    long_name:  Area of each face.

Now calculate the area again with the `Grid.compute_face_areas` function using arguments: quadrature_rule "gaussian" and order 4

In [6]:
all_face_areas, all_face_jacobians = ugrid.compute_face_areas(
    quadrature_rule="gaussian", order=4
)
g4_area = all_face_areas.sum()
g4_area

np.float64(12.566370614359112)

Now we compare the values with actual know value and report error for each of the three cases above.

In [7]:
actual_area = 4 * np.pi
diff_t4_area = np.abs(t4_area - actual_area)
diff_t1_area = np.abs(t1_area - actual_area)
diff_g4_area = np.abs(g4_area - actual_area)

diff_t1_area, diff_t4_area, diff_g4_area

(np.float64(0.005033379360810386),
 np.float64(3.1938185429680743e-10),
 np.float64(6.039613253960852e-14))

As we can see, it is clear that the Gaussian Quadrature Rule with Order 4 is the most accurate, and the Triangular Quadrature Rule with Order 1 is the least accurate.


## 4. Calculate Area of a Single Triangle in Cartesian Coordinates

For this section, we create a single triangle face with 3 vertices. By default, in `uxarray`, we assume that the coordinate system is spherical (lat / lon), however if you want to use cartesian coordinates, you must pass through `latlon = False` into the `Grid` constructor.

Assume the units in meters - this is a big triangle!

In [8]:
verts = [
    [
        [0.57735027, -5.77350269e-01, -0.57735027],
        [0.57735027, 5.77350269e-01, -0.57735027],
        [-0.57735027, 5.77350269e-01, -0.57735027],
    ]
]

# load our vertices into a UXarray Grid object
vgrid = ux.open_grid(verts, latlon=False)

vgrid

<uxarray.Grid>
Original Grid Type: Face Vertices
Grid Dimensions:
  * n_node: 3
  * n_face: 1
  * n_max_face_nodes: 3
  * n_nodes_per_face: (1,)
Grid Coordinates (Spherical):
Grid Coordinates (Cartesian):
  * node_x: (3,)
  * node_y: (3,)
  * node_z: (3,)
Grid Connectivity Variables:
  * face_node_connectivity: (1, 3)
Grid Descriptor Variables:
  * n_nodes_per_face: (1,)

In [9]:
vgrid.calculate_total_face_area()

np.float64(1.0719419938548218)

Additionally, if you are using a unit other than meters, you can update the units as follows

In [10]:
# Calculate the area of the triangle
area_gaussian = vgrid.calculate_total_face_area(quadrature_rule="gaussian", order=5)
area_gaussian

np.float64(1.0475702709086991)

## 5. Calculate Area from Multiple Faces in Spherical Coordinates

Similar to above, we can construct a `Grid` object with multiple faces by passing through a set of vertices. Here we define 3 six-sided faces.

In [11]:
faces_verts_ndarray = np.array(
    [
        np.array(
            [
                [150, 10, 0],
                [160, 20, 0],
                [150, 30, 0],
                [135, 30, 0],
                [125, 20, 0],
                [135, 10, 0],
            ]
        ),
        np.array(
            [
                [125, 20, 0],
                [135, 30, 0],
                [125, 60, 0],
                [110, 60, 0],
                [100, 30, 0],
                [105, 20, 0],
            ]
        ),
        np.array(
            [
                [95, 10, 0],
                [105, 20, 0],
                [100, 30, 0],
                [85, 30, 0],
                [75, 20, 0],
                [85, 10, 0],
            ]
        ),
    ]
)

We want our units to be spherical, so we pass through `latlon=True`. Additionally, if `latlon` is not passed through, it will default to spherical coordinates.

In [12]:
verts_grid = ux.open_grid(faces_verts_ndarray, latlon=True)

verts_grid

<uxarray.Grid>
Original Grid Type: Face Vertices
Grid Dimensions:
  * n_node: 14
  * n_face: 3
  * n_max_face_nodes: 6
  * n_nodes_per_face: (3,)
Grid Coordinates (Spherical):
  * node_lon: (14,)
  * node_lat: (14,)
Grid Coordinates (Cartesian):
Grid Connectivity Variables:
  * face_node_connectivity: (3, 6)
Grid Descriptor Variables:
  * n_nodes_per_face: (3,)

In [13]:
area, jacobian = verts_grid.compute_face_areas()
area

array([0.14323746, 0.25118746, 0.12141312])

## 6. Area Correction

The correction, \(A\), is calculated using the following formula:

```{math}
A = 2arctan(z(x_{1}y_{2} - x_{2} y_{1}) / {x_{1}^2 + y_{1}^2 + x_{1} x_{2} + y_{1} y_{2}) - z (x_{1} y_{2} - x_{2} y_{1} / x_{1} x_{2} + y_{1} y_{2}
```

### Where:
<ul>

<li>(x<sub>1</sub>, y<sub>1</sub>, z) are the Cartesian coordinates of the first node.</li>

<li>(x<sub>2</sub>, y<sub>2</sub>, z) are the Cartesian coordinates of the second node (note that the z coordinate is the same for both nodes).</li>

</ul>

### Assumptions:
- This formula assumes that the input coordinates \((x_1, y_1)\) and \((x_2, y_2)\) are normalized (i.e., they lie on the unit sphere).


For the same large triangle used in **Section 4**, we calculate the area correction term when an edge lies along the line of constant latitude.

### The following code:
- Creates a spherical rectangle and calculates the exact area using the spherical excess formula.
- Plots the rectangle, it has two lines of constant lattitude.
- A mesh is formed with uxarray and area is calculated
- Area is again calculated with `correct_area` flag set to `True`

In [8]:
def theoretical_spherical_rectangle_area(lons, lats, radius=1):
    """
    Calculate the area of a spherical rectangle on a sphere.

    Parameters:
        lons (np.ndarray): Longitudes of the rectangle's corners (in degrees).
        lats (np.ndarray): Latitudes of the rectangle's corners (in degrees).
        radius (float): Radius of the sphere (default is unit sphere).

    Returns:
        float: Area of the spherical rectangle in steradians (or square units if radius is provided).
    """
    # Convert degrees to radians
    lons_rad = np.radians(lons)
    lats_rad = np.radians(lats)

    # Compute longitude and latitude differences
    delta_lambda = abs(lons_rad[0] - lons_rad[2])  # Assuming rectangular shape
    sin_phi_diff = abs(np.sin(lats_rad[0]) - np.sin(lats_rad[2]))

    # Calculate area
    area = radius**2 * delta_lambda * sin_phi_diff
    return area


# Define nodes
node_lon = np.array([90.0, 90.0, 10.0, 10.0])  # Longitudes in degrees
node_lat = np.array([80.0, 10.0, 10.0, 80.0])  # Latitudes in degrees

# Calculate the area
area_steradians = theoretical_spherical_rectangle_area(node_lon, node_lat)
print(
    f"The area of the spherical rectangle is approximately {area_steradians:.6f} steradians"
)

# If Earth's radius is used (R ≈ 6371 km):
earth_radius_km = 6371
area_km2 = theoretical_spherical_rectangle_area(
    node_lon, node_lat, radius=earth_radius_km
)
print(
    f"The area of the spherical rectangle is approximately {area_km2:.2f} square kilometers"
)

The area of the spherical rectangle is approximately 1.132592 steradians
The area of the spherical rectangle is approximately 45971520.05 square kilometers


In [10]:
# Define face-node connectivity and create/plot a uxarray object face
face_node_connectivity = np.array([[0, 1, 2, 3]])

face = ux.Grid.from_topology(
    node_lon=node_lon,
    node_lat=node_lat,
    face_node_connectivity=face_node_connectivity,
    fill_value=-1,
)
face.plot(backend="bokeh")

:Path   [Longitude,Latitude]

In [13]:
area_no_correction = face.compute_face_areas(quadrature_rule="gaussian", order=8)

In [17]:
corrected_area = face.compute_face_areas(
    quadrature_rule="gaussian", order=8, correct_area=True
)

Check if edge passes through pole:  False
6.030208312509488e-17 0.984807753012208 0.9698463103929541 0.17101007166283433 0.17364817766693033 correction: 0.04692118879641144


calculated correction:  0.04692118879641144
Edge is in the  Northern  hemisphere.
For Node 1  [6.03020831e-17 9.84807753e-01 1.73648178e-01] 
 and Node 2 [0.96984631 0.17101007 0.17364818] 
CORRECTION 0.04692118879641144
Check if edge passes through pole:  False
0.1710100716628344 0.030153689607045817 1.0632884247878861e-17 0.17364817766693041 0.984807753012208 correction: 0.006156712978357515


calculated correction:  0.006156712978357515
Edge is in the  Northern  hemisphere.
For Node 1  [0.17101007 0.03015369 0.98480775] 
 and Node 2 [1.06328842e-17 1.73648178e-01 9.84807753e-01] 
CORRECTION -0.006156712978357515
AREA Before Correction 1.0918281804990866
AREA After Correction 1.1325926563171405


In [20]:
# Calculate the percentage difference between the corrected and uncorrected areas
percentage_difference = (
    (corrected_area[0][0] - area_no_correction[0][0]) / area_no_correction[0][0] * 100
)

print(
    f"The percentage difference between the corrected and uncorrected areas is {percentage_difference:.2f}%"
)

The percentage difference between the corrected and uncorrected areas is 3.73%
